In [1]:
import pandas as pd
import requests
import json

In [2]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Chave pública
numero_processo = int(input("Entre com o número do processo: ")) # Exemplos: 15000156520218260451, 15072172520238260451

payload = json.dumps({
  "query": {
    "match": {
      "numeroProcesso": numero_processo
    }
  }
})

headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
dados_dict = response.json() # <class 'dict'>
print(dados_dict)

{'took': 1246, 'timed_out': False, '_shards': {'total': 20, 'successful': 20, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 14.289829, 'hits': [{'_index': 'api_publica_tjsp', '_type': '_doc', '_id': 'TJSP_279_G1_59205_15059133020198260451', '_score': 14.289829, '_source': {'classe': {'codigo': 279, 'nome': 'Inquérito Policial'}, 'numeroProcesso': '15059133020198260451', 'sistema': {'codigo': 3, 'nome': 'SAJ'}, 'formato': {'codigo': 1, 'nome': 'Eletrônico'}, 'tribunal': 'TJSP', 'dataHoraUltimaAtualizacao': '2023-08-30T08:09:33.266Z', 'grau': 'G1', '@timestamp': '2023-10-17T02:31:26.730Z', 'dataAjuizamento': '2019-06-24T16:40:17.000Z', 'movimentos': [{'complementosTabelados': [{'codigo': 2, 'valor': 2, 'nome': 'sorteio', 'descricao': 'tipo_de_distribuicao_redistribuicao'}], 'codigo': 26, 'nome': 'Distribuição', 'dataHora': '2019-06-24T16:40:43.000Z'}, {'complementosTabelados': [{'codigo': 19, 'valor': 57, 'nome': 'Petição (outras)', 'descricao

In [3]:
# Descobrindo código e órgão julgador 
codigo = dados_dict['hits']['hits'][0]['_source']['orgaoJulgador']['codigo']
orgao_julgador = dados_dict['hits']['hits'][0]['_source']['orgaoJulgador']['nome']

print (f'Código: {codigo} - Órgão julgador: {orgao_julgador}')

Código: 59205 - Órgão julgador: 04 CRIMINAL DE PIRACICABA


In [4]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
api_key = 'APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==' # Chave pública
orgao_julgador = 10071 # 10066 10065 10067 59205 10071

payload = json.dumps({
  "size": 10000,
  "query": {
    "match": {"orgaoJulgador.codigo": orgao_julgador
    #"match": {"orgaoJulgador.codigoMunicipioIBGE": 3538709             
    }
  },
  "sort": [{"dataAjuizamento": {"order": "desc"}}] # ou asc
})

headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dados_dict = json.loads(response.text)

In [5]:
dados_dict['hits']['hits'][0]

{'_index': 'api_publica_tjsp',
 '_type': '_doc',
 '_id': 'TJSP_386_G1_10071_00071373820238260451',
 '_score': None,
 '_source': {'classe': {'codigo': 386, 'nome': 'Execução da Pena'},
  'numeroProcesso': '00071373820238260451',
  'sistema': {'codigo': 3, 'nome': 'SAJ'},
  'formato': {'codigo': 1, 'nome': 'Eletrônico'},
  'tribunal': 'TJSP',
  'dataHoraUltimaAtualizacao': '2023-08-30T08:23:41.169Z',
  'grau': 'G1',
  '@timestamp': '2023-10-17T02:04:12.283Z',
  'dataAjuizamento': '2023-07-31T16:13:40.000Z',
  'movimentos': [{'complementosTabelados': [{'codigo': 2,
      'valor': 1,
      'nome': 'competência exclusiva',
      'descricao': 'tipo_de_distribuicao_redistribuicao'}],
    'codigo': 26,
    'nome': 'Distribuição',
    'dataHora': '2023-07-31T16:20:52.000Z'},
   {'codigo': 11383,
    'nome': 'Ato ordinatório',
    'dataHora': '2023-08-03T09:35:19.000Z'},
   {'complementosTabelados': [{'codigo': 4,
      'valor': 79,
      'nome': 'Ofício',
      'descricao': 'tipo_de_documento'}

In [6]:
processos = []

for processo in dados_dict['hits']['hits']:
  numero_processo = processo['_source']['numeroProcesso']
  grau = processo['_source']['grau']
  classe = processo['_source']['classe']['nome']
  assuntos = processo['_source']['assuntos'] # Pode ter mais de um
  data_ajuizamento = processo['_source']['dataAjuizamento']
  ultima_atualizacao = processo['_source']['dataHoraUltimaAtualizacao']
  formato = processo['_source']['formato']['nome']
  codigo = processo['_source']['orgaoJulgador']['codigo']
  orgao_julgador = processo['_source']['orgaoJulgador']['nome']
  municipio = processo['_source']['orgaoJulgador']['codigoMunicipioIBGE']
  try:
    movimentos = processo['_source']['movimentos']
  except:
    movimentos = []

  processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, formato, \
                    codigo, orgao_julgador, municipio, grau, assuntos, movimentos])

df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao', \
                      'formato', 'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos'])   

df.sample(5)               

,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos
741,00036760220238260502,Agravo de Execução Penal,2023-03-24T11:14:19.000Z,2023-07-20T03:32:11.593Z,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,"[{'codigo': 10635, 'nome': 'Progressão de Regi...","[{'complementosTabelados': [{'codigo': 19, 'va..."
9309,00028048920208260502,Execução da Pena,2020-02-28T15:16:53.000Z,2023-07-20T03:30:28.402Z,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,"[{'codigo': 7791, 'nome': 'Pena Privativa de L...","[{'complementosTabelados': [{'codigo': 2, 'val..."
9620,15121533520198260451,Inquérito Policial,2019-12-19T11:04:30.000Z,2023-07-20T02:08:59.738Z,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,"[{'codigo': 3372, 'nome': 'Homicídio Qualifica...","[{'complementosTabelados': [{'codigo': 2, 'val..."
199,10128654320238260451,Execução de Pena de Multa,2023-07-01T15:48:22.000Z,2023-08-30T08:21:03.084Z,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,"[{'codigo': 7792, 'nome': 'Pena de Multa'}]","[{'complementosTabelados': [{'codigo': 2, 'val..."
9216,00037939520208260502,Execução da Pena,2020-03-26T15:23:13.000Z,2023-07-20T03:30:31.627Z,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,"[{'codigo': 10635, 'nome': 'Progressão de Regi...","[{'complementosTabelados': [{'codigo': 2, 'val..."


In [7]:
def conversor_data(data_str):
    return pd.to_datetime(data_str).tz_convert('America/Sao_Paulo')


def gera_lista_assuntos(assuntos_do_df):
    lst_assuntos=[]
    for assunto in assuntos_do_df:
        try:
            lst_assuntos.append(assunto.get('nome'))
        except:
            lst_assuntos.append('')

    return lst_assuntos


def gera_tuplas_movimentos(movimentos):
    lst_movimentos_final =[]
    for movimento in movimentos:
        #lst_movimentos_final.append(( movimento.get('nome'), conversor_data(movimento.get('dataHora'))))
        lst_movimentos_final.append(( movimento.get('codigo'), movimento.get('nome'), movimento.get('dataHora')))
    return lst_movimentos_final

In [8]:
df['assuntos'] = df['assuntos'].apply(gera_lista_assuntos)
df['movimentos'] = df['movimentos'].apply(gera_tuplas_movimentos)
df['data_ajuizamento'] = df['data_ajuizamento'].apply(conversor_data)
df['ultima_atualizacao'] = df['ultima_atualizacao'].apply(conversor_data)
df['movimentos']= df['movimentos'].apply(lambda x: sorted(x, key=lambda tup: tup[2], reverse=False))
df.sample(5)

,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos
2652,10121295920228260451,Execução de Medidas Alternativas no Juízo Comum,2022-07-01 07:49:22-03:00,2023-07-19 23:40:43.002000-03:00,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,[Acordo de Não Persecução Penal],"[(26, Distribuição, 2022-07-01T16:12:57.000Z),..."
7447,10194338020208260451,Execução de Pena de Multa,2020-11-12 14:07:51-03:00,2023-07-19 23:26:11.488000-03:00,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,[Pena de Multa],"[(26, Distribuição, 2020-11-13T10:41:50.000Z),..."
9493,70000172420208260451,Execução da Pena,2020-01-26 21:00:00-03:00,2022-08-03 15:00:26.962000-03:00,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,[Pena Privativa de Liberdade],[]
5820,10114664720218260451,Execução de Pena de Multa,2021-06-23 11:30:52-03:00,2023-08-30 05:13:08.278000-03:00,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,[Pena de Multa],"[(26, Distribuição, 2021-06-23T17:19:03.000Z),..."
1644,00100684820228260451,Execução da Pena,2022-10-07 15:23:23-03:00,2023-07-19 23:43:55.045000-03:00,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,[Prestação de Serviços à Comunidade],"[(26, Distribuição, 2022-10-07T18:22:44.000Z),..."


In [9]:
df.to_csv('processos.csv', sep=',', header=True, index=False)

# Análises

In [10]:
with open('resultado.txt', 'w') as file:
    for index, row in df.iterrows():

        file.write(f"Autos n. {row['numero_processo']}, Classe: {row['classe']}\n") 
        file.write(f"Ajuizamento: {row['data_ajuizamento']}\n")
        file.write(f"Assuntos: {row['assuntos']}\n\n")
        for movimento in row['movimentos']:  
            file.write(f"{str(movimento[2])} | Cód: {str(movimento[0])} | Mov.: {movimento[1]} \n")
        file.write('\n-----------------------------------------------------------------------------\n\n')